<a href="https://colab.research.google.com/github/mamadoz79/NLP-CA/blob/main/word2vec_CA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install docx2txt
!pip install hazm

  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3980 sha256=e3cb1ee0602d1e502d40aa97384d127fb898659d83f34de5b9714f708d46663b
  Stored in directory: /root/.cache/pip/wheels/b7/20/b2/473e3aea9a0c0d3e7b2f7bd81d06d0794fec12752733d1f3a8
Successfully built docx2txt
     |████████████████████████████████| 316 kB 7.5 MB/s 
     |████████████████████████████████| 233 kB 56.7 MB/s 
     |████████████████████████████████| 1.4 MB 44.6 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=df253a61552beef5c62962d179484429e8ec17738dae03f459738c501dfc4cdc
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154069 sha256=f7cbc709ae01f16ff7df4210b87092c301097b398ed8df8968ccbc630ac134b4
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully 

In [ ]:
from collections import defaultdict
import docx2txt
import re
import numpy as np
import hazm
import re

In [ ]:
text = docx2txt.process("/3.docx")

In [ ]:
text = hazm.Normalizer().normalize(text)

In [ ]:
def clean_str(text):
    C = ["ئ", " ", "ء", "ؤ", "لآ", "آ", "ر", "لا", "ى", "ة", "و", "ز", "ظ", "ش", "س", "ي", "ب", "ل", "ا", "لأ", "أ",
         "لإ", "إ", "ت", "ن", "م", "ك", "ط", "ض", "ص", "ث", "ق", "ف", "غ", "ع", "ه", "خ", "ح", "ج", "د", "ذ"]
    rx = '[^' + re.escape(''.join(C)) + ']'
    text = re.sub(rx, ' ', text)

    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel, "", text)

    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)

    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    text = text.strip()

    return text

In [ ]:
for i in text.split():
    i = clean_str(i)

In [ ]:
text = re.sub('[0-9۰-۹]+', ' ', text)
text = re.sub('\([0-9۰-۹]+\)', ' ', text)
text = re.sub('[\\n]+', ' ', text)
text = re.sub('[ ]{2,}', ' ', text)

In [ ]:
d = defaultdict(int)
for i in text.split():
    d[i] += 1

In [ ]:
def word2onehot(word, v_count, word_index=d):
    word_vec = [0 for i in range(0, v_count)]
    word_index = word_index[word]
    word_vec[word_index] = 1
    return word_vec

In [ ]:
res = word2onehot(d[1], len(d.items()))

In [ ]:
class word2vec():
    def __init__ (self):
        self.n = settings['n']
        self.eta = settings['learning_rate']
        self.epochs = settings['epochs']
        self.window = settings['window_size']
        pass
    
    
    def generate_training_data(self, settings, corpus):

        word_counts = defaultdict(int)
        for row in corpus:
            for word in row:
                word_counts[word] += 1

        self.v_count = len(word_counts.keys())

        self.words_list = sorted(list(word_counts.keys()),reverse=False)
        self.word_index = dict((word, i) for i, word in enumerate(self.words_list))
        self.index_word = dict((i, word) for i, word in enumerate(self.words_list))

        training_data = []
        for sentence in corpus:
            sent_len = len(sentence)

            for i, word in enumerate(sentence):
                
                w_target = self.word2onehot(sentence[i])

                w_context = []
                for j in range(i-self.window, i+self.window+1):
                    if j!=i and j<=sent_len-1 and j>=0:
                        w_context.append(self.word2onehot(sentence[j]))
                training_data.append([w_target, w_context])
        return np.array(training_data)


    def softmax(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)

    def word2onehot(self, word):
        word_vec = [0 for i in range(0, self.v_count)]
        word_index = self.word_index[word]
        word_vec[word_index] = 1
        return word_vec

    def forward_pass(self, x):
        h = np.dot(self.w1.T, x)
        u = np.dot(self.w2.T, h)
        y_c = self.softmax(u)
        return y_c, h, u
                

    def backprop(self, e, h, x):
        dl_dw2 = np.outer(h, e)  
        dl_dw1 = np.outer(x, np.dot(self.w2, e.T))

        self.w1 = self.w1 - (self.eta * dl_dw1)
        self.w2 = self.w2 - (self.eta * dl_dw2)
        pass


    def train(self, training_data):
        self.w1 = np.random.uniform(-0.8, 0.8, (self.v_count, self.n))
        self.w2 = np.random.uniform(-0.8, 0.8, (self.n, self.v_count))   
        
        for i in range(0, self.epochs):

            self.loss = 0

            for w_t, w_c in training_data:

                y_pred, h, u = self.forward_pass(w_t)
                EI = np.sum([np.subtract(y_pred, word) for word in w_c], axis=0)

                self.backprop(EI, h, w_t)
                self.loss += -np.sum([u[word.index(1)] for word in w_c]) + len(w_c) * np.log(np.sum(np.exp(u)))
                
            print ('EPOCH:',i, 'LOSS:', self.loss)
        pass


    def word_vec(self, word):
        w_index = self.word_index[word]
        v_w = self.w1[w_index]
        return v_w


    def vec_sim(self, vec, top_n):

        word_sim = {}
        for i in range(self.v_count):
            v_w2 = self.w1[i]
            theta_num = np.dot(vec, v_w2)
            theta_den = np.linalg.norm(vec) * np.linalg.norm(v_w2)
            theta = theta_num / theta_den

            word = self.index_word[i]
            word_sim[word] = theta

        words_sorted = sorted(word_sim.items(), key=lambda word, sim :sim, reverse=True)

        for word, sim in words_sorted[:top_n]:
            print (word, sim)
            
        pass
    def word_sim(self, word, top_n):
        
        w1_index = self.word_index[word]
        v_w1 = self.w1[w1_index]

        word_sim = {}
        for i in range(self.v_count):
            v_w2 = self.w1[i]
            theta_num = np.dot(v_w1, v_w2)
            theta_den = np.linalg.norm(v_w1) * np.linalg.norm(v_w2)
            theta = theta_num / theta_den

            word = self.index_word[i]
            word_sim[word] = theta

        words_sorted = sorted(word_sim.items(), key=lambda word, sim : sim, reverse=True)

        for word, sim in words_sorted[:top_n]:
            print (word, sim)
            
        pass

In [ ]:
settings = {}
settings['n'] = 5                   
settings['window_size'] = 2       
settings['min_count'] = 0          
settings['epochs'] = 100         
settings['neg_samp'] = 10           
settings['learning_rate'] = 0.01    
np.random.seed(0)                   

w2v = word2vec()
+c = [i.split() for i in text.split('.')]